Changing the string representation of instances

In [5]:
class Pair:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __repr__(self):
        return 'Pair({},{})'.format(self.x,self.y)
    def __str__(self):
        return '({},{})'.format(self.x,self.y)

In [6]:
p=Pair(3,4)
p

Pair(3,4)

In [7]:
print p

(3,4)


In [8]:
print 'p is {!r}'.format(p)

p is Pair(3,4)


In [9]:
print 'p is {}'.format(p)

p is (3,4)


Customizing string formatting

In [15]:
_formats={
        'ymd':'{d.year}-{d.month}-{d.day}',
        'mdy':'{d.month}/{d.day}/{d.year}',
        'dmy':'{d.day}/{d.month}/{d.year}'
        }
class Date:
    def __init__(self,year,month,day):
        self.year=year
        self.month=month
        self.day=day
    def __format__(self,code):
        if code=='':
            code='ymd'
        fmt=_formats[code]
        return fmt.format(d=self)

In [16]:
d=Date(2012,12,9)
format(d)

'2012-12-9'

In [17]:
format(d,'mdy')

'12/9/2012'

Calling a method on a parent class

In [35]:
#http://stackoverflow.com/questions/1713038/super-fails-with-error-typeerror-argument-1-must-be-type-not-classobj
class A(object):
    def spam(self):
        print 'A.spam'

class B(A):
    def spam(self):
        print 'B.spam'
        print super(B,self).spam()

In [36]:
a=A()
b=B()

In [37]:
a.spam()

A.spam


In [39]:
b.spam()

B.spam
A.spam
None


访问限制，__

In [2]:
class Student(object):
    def __init__(self,name,score):
        self._name=name
        self.__score=score
    def print_score(self):
        print '%s:%s'(self._name,self.__score)

In [3]:
zhanghaitao=Student('zhanghaitao',94)
zhanghaitao._name

'zhanghaitao'

In [4]:
zhanghaitao.__score

AttributeError: 'Student' object has no attribute '__score'

如果有要允许外部代码修改score怎么办？可以给Student类增加set_score方法：

In [5]:
class Student(object):
    def __init__(self,name,score):
        self._name=name
        self.__score=score
    def print_score(self):
        print '%s:%s'(self._name,self.__score)
    def set_score(self,score):
        self.__score=score

# 继承和多态

In [6]:
class Animal(object):
    def run(self):
        print 'Animal is running...'

class Dog(Animal):
    pass

class Cat(Animal):
    pass

In [7]:
dog=Dog()
cat=Cat()

In [8]:
dog.run()

Animal is running...


In [9]:
cat.run()

Animal is running...


of course,you can add some method for the subclass

In [10]:
class Dog(Animal):
    def run(self):
        print 'Dog is running...'

class Cat(Animal):
    def run(self):
        print 'Cat is running...'


In [11]:
dog=Dog()
cat=Cat()

In [12]:
dog.run()

Dog is running...


In [13]:
cat.run()

Cat is running...


当子类和父类都存在相同的run()方法是，我们说，子类的run()覆盖了父类的run(),
在运行的时候，总是会调用子类的run()。这样，我们就获得了继承的另一个好处：多态。


当我们定义一个class的时候，实际上就定义了一种数据类型。我们定义的数据类型
和python自带的数据类型，比如str，dict没有两样。

In [16]:
a=list()
b=Animal()
c=Dog()

In [17]:
isinstance(a,list)

True

In [18]:
isinstance(c,Dog)

True

In [19]:
isinstance(c,Animal)

True

# 使用__slots__

In [20]:
class Student(object):
    pass

In [21]:
#给实例绑定一个属性：
s=Student()
s.name='Micheal'
print s.name

Micheal


In [22]:
#还可以给实例绑定一个方法：
def set_age(self,age):
    self.age=age

from types import MethodType
s.set_age=MethodType(set_age,s,Student)
s.set_age(25)
s.age

25

In [24]:
#但是，给一个实例绑定的方法，对另一个实例是不起作用的
s2=Student()
s2.age

AttributeError: 'Student' object has no attribute 'age'

In [25]:
#为了给所有实例都绑定方法，可以给class绑定方法
def set_score(self,score):
    self.score=score
Student.set_score=MethodType(set_score,None,Student)

In [26]:
s.set_score(100)
s.score

100

In [27]:
s2.set_score(98)
s2.score

98

# 使用@property

In [30]:
#在绑定属性时候，如果我们直接把属性暴露出去，虽然写起来很简单，但是
#没办法检查参数，导致可以把成绩随便更改：
s=Student()
s.score=9999

In [31]:
#为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个
#get_score()来获取成绩，这样，在set_score()里就可以检查参数：
class Student(object):
    def get_score(self):
        return self._score
    
    def set_score(self,score):
        if not isinstance(value,int):
            raise ValueError('score must be an integer!')
        if value<0 or value>100:
            raise ValueError('score must be between 1~100!')
        self._score=value


In [33]:
#python 内置的@property装饰器就是负责把一个方法变成属性调用的：
class Student(object):
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('score must be int!')
        if value<0 or value>100:
            raise ValueError('score must be between 0~100')
        self._score=value
#@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需
#加上@property就可以了，此时，@property本省有创建了另一个装饰器@score.setter
#负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：
s=Student()
s.score=60
s.score

60

In [34]:
s.score=9999

ValueError: score must be between 0~100

In [36]:
#还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：
class Student(object):
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth=value
    
    @property
    def age(self):
        return 2014-self._birth
    

# 定制类

In [37]:
class Student(object):
    def __init__(self,name):
        self.name=name
    def __str__(self):
        return 'Student object (name={})'.format(self.name)
    __repr__=__str__

In [39]:
#如果一个类想被用于for...in 循环，类似list或tuple那样，就必须实现一个
#__iter__()方法，该方法返回一个迭代对象，然后，python的for循环就会不断调用
#该迭代对象的next()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。
class Fib(object):
    def __init__(self):
        self.a,self.b=0,1
    
    def __iter__(self):
        return self
    
    def next(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>10000:
            raise StopIteration()
        return self.a

In [40]:
for n in Fib():
    print n

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765


In [41]:
#要表现的像list那样按照下标取元素，需要实现__getitem__()方法
class Fib(object):
    def __getitem__(self,n):
        a,b=1,1
        for x in range(n):
            a,b=b,a+b
        return a

In [42]:
f=Fib()
f[0]

1

In [43]:
f[4]

5

In [44]:
f[100]

573147844013817084101L

In [58]:
#如果想要像list那样切片，需要对传入的参数做判断：
class Fib(object):
    def __getitem__(self,n):
        if isinstance(n,int):
            a,b=1,1
            for i in range(n):
                a,b=b,a+b
            return a
        if isinstance(n,slice):
            start=n.start
            stop=n.stop
            a,b=1,1
            L=[]
            for x in range(stop):
                if x>=start:
                    L.append(a)
                a,b=b,a+b
            return L

In [59]:
f=Fib()
f[0:10]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

# __getattr__

getattr(object, name[, default])
Return the value of the named attribute of object. name must be a string. If the string is the name of one of the object's attributes, the result is the value of that attribute. For example, getattr(x, 'foobar') is equivalent to x.foobar. If the named attribute does not exist, default is returned if provided, otherwise AttributeError is raised.


In [64]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='score':
            return 99
        if attr=='address':
            return None
        raise AttributeError('Student object has no attribute {}'.format(attr))
        

In [65]:
s=Student()

In [66]:
s.score

99

In [67]:
s.age

AttributeError: Student object has no attribute age

In [69]:
print s.address

None


# Call

###### 一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。任何类，只需要定义一个__call__()方法，就可以直接对实例进行调用。

In [71]:
class Student(object):
    def __init__(self,name):
        self.name=name
    def __call__(self):
        print 'my name is {}'.format(self.name)

In [72]:
s=Student('Michael')
s()

my name is Michael
